## Loading

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import datasets
import pandas as pd
import seaborn as sns
from bertviz import model_view, head_view


from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from transformers import (
    ElectraTokenizerFast,
    ElectraForSequenceClassification,
    ElectraConfig,
)

from helpers import prepare_dataset_nli

NUM_PREPROCESSING_WORKERS = 2
pretrained_model = "google/electra-small-discriminator"
train_path = './trains/'
eval_path= './evals/'
dataset_name = './snli_classic/'

# dataset_name = 'snli_classic/'

# dataset_name = './snli_hyponly/'


## Device check

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

There are 8 GPU(s) available.
Device name: Tesla K80


## Model

In [7]:
# Train Params
lr = 2e-5
num_epochs = 1
batch_size = 100*torch.cuda.device_count()
# # Load the model
# config = ElectraConfig.from_pretrained(pretrained_model)
# config.gradient_checkpointing = False
# config.use_cache = True
# # config.problem_type = "multi_label_classification"
# config.num_labels = 3

# model = ElectraForSequenceClassification.from_pretrained(
#     "google/electra-small-discriminator",
#     config=config
# )

# # Move model to device
# # Create the Optimizer
optimizer = optim.AdamW(model.parameters(), lr=lr)
# model = torch.nn.DataParallel(model, device_ids=[0, 1, 2, 3, 4, 5 ,6 ,7])
# model.to(device)


## Dataset loading and preprocessing

In [5]:
dataset = datasets.load_dataset('snli')
dataset = dataset.filter(lambda ex: ex['label'] != -1)

if dataset_name == './snli_hyponly/':
    df = dataset['train'].to_pandas()
    df['premise'] = ' '
    dataset = datasets.DatasetDict({'train': datasets.Dataset.from_pandas(df)}) 

tokenizer = ElectraTokenizerFast.from_pretrained(pretrained_model)
prepare_train_dataset = prepare_eval_dataset = lambda exs: prepare_dataset_nli(exs, tokenizer, 128)

train_dataset = dataset['train']
train_dataset_featurized = train_dataset.map(
    prepare_train_dataset,
    batched=True,
    num_proc=NUM_PREPROCESSING_WORKERS,
    remove_columns=train_dataset.column_names
)

config = ElectraConfig.from_pretrained(dataset_name + 'model')
model = ElectraForSequenceClassification.from_pretrained(dataset_name + 'model', config=config)
tokenizer = ElectraTokenizerFast.from_pretrained(dataset_name + 'model')
model = torch.nn.DataParallel(model, device_ids=[0, 1, 2, 3, 4, 5 ,6 ,7])
model.to(device)

train_loader = DataLoader(train_dataset_featurized, batch_size=batch_size, shuffle=True, pin_memory=True)

Reusing dataset snli (/home/ml/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)
Loading cached processed dataset at /home/ml/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b/cache-4eacbddb81939caf.arrow
Loading cached processed dataset at /home/ml/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b/cache-ab24f4c6f0ce7e93.arrow
Loading cached processed dataset at /home/ml/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b/cache-a1c734a8e7f96896.arrow


## Train

In [8]:
model.train()

global_step = 0

# You can choose to do your own loss calculation if you'd like, but I'm keeping things simple here
loss = None

for epoch in range(num_epochs):
    pragati = tqdm(train_loader, desc=f'Epoch {epoch}', leave=False)

    for n, batch in enumerate(pragati):
    
        model.zero_grad()

        # Change labels depending on whether you're doing NER/Parsing (token classification)
        # or sentiment classification (sequence classification)
        # ELECTRA Page linked here: https://huggingface.co/transformers/model_doc/electra.html
        labels = batch.pop('labels')
        labels = labels.to(device)
        for k in batch:
            batch[k] = torch.stack(batch[k], dim=1).to(device)
        model_output = model(**batch, labels=labels)
        loss = model_output.loss
        loss.sum().backward()
        optimizer.step()
        global_step += 1

        if n % 20 == 0:
            pragati.set_description(f'Epoch: {epoch} | Step: {global_step} | Loss: {loss.mean():.2f}')

    print(f'Epoch {epoch} | Global Step: {global_step} | Loss : {loss.mean():.2f}')

print(f'Completed Training at Epoch {epoch} | Global Step: {global_step} | Loss : {loss.mean():.2f}')

tokenizer.save_pretrained(dataset_name + 'model')
model.module.save_pretrained(dataset_name + 'model')
config.save_pretrained(dataset_name + 'model')

Epoch 0:   0%|          | 0/687 [00:00<?, ?it/s]

Epoch 0 | Global Step: 687 | Loss : 0.72
Completed Training at Epoch 0 | Global Step: 687 | Loss : 0.72


## Evaluation

In [29]:
# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
dataset = datasets.load_dataset('snli')
dataset = dataset.filter(lambda ex: ex['label'] != -1)

if dataset_name == './snli_hyponly/':
    df = dataset['validation'].to_pandas()
    df['premise'] = ' '
    dataset = datasets.DatasetDict({'validation': datasets.Dataset.from_pandas(df)}) 
else:
    df_1 = dataset['validation'].to_pandas()
    df_2 = dataset['test'].to_pandas()
    df = pd.concat([df_1, df_2], axis=0).reset_index(drop=True)
    dataset = datasets.DatasetDict({'validation': datasets.Dataset.from_pandas(df)}) 




# dataset = datasets.load_dataset('snli')
# dataset = dataset.filter(lambda ex: ex['label'] != -1)

batch_size = 1 #36*torch.cuda.device_count()
# config = ElectraConfig.from_pretrained(pretrained_model)
# config.gradient_checkpointing = False
# config.use_cache = True
# # config.problem_type = "multi_label_classification"
# config.num_labels = 3

# config = ElectraConfig.from_pretrained(dataset_name + 'model')
model = ElectraForSequenceClassification.from_pretrained(dataset_name + 'model')#,# config=config)
tokenizer = ElectraTokenizerFast.from_pretrained(dataset_name + 'model')

Reusing dataset snli (/home/ml/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)
Loading cached processed dataset at /home/ml/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b/cache-4eacbddb81939caf.arrow
Loading cached processed dataset at /home/ml/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b/cache-ab24f4c6f0ce7e93.arrow
Loading cached processed dataset at /home/ml/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b/cache-a1c734a8e7f96896.arrow


In [31]:
prepare_train_dataset = prepare_eval_dataset = lambda exs: prepare_dataset_nli(exs, tokenizer, 128)

eval_dataset = dataset['validation']
eval_dataset_featurized = eval_dataset.map(
    prepare_eval_dataset,
    batched=False,
    num_proc=NUM_PREPROCESSING_WORKERS,
    remove_columns=['label'] #eval_dataset.column_names
)
# eval_loader = DataLoader(eval_dataset_featurized, batch_size=batch_size, shuffle=True, pin_memory=True)

In [32]:
# loss = nn.CrossEntropyLoss()
model_keys = ['input_ids', 'token_type_ids', 'attention_mask']
model.eval()
model.to(device)
eval_samples = []
attentions = []
for n, sample in tqdm(enumerate(eval_dataset_featurized), total=len(eval_dataset_featurized)):

    # Change labels depending on whether you're doing NER/Parsing (token classification)
    # or sentiment classification (sequence classification)
    # ELECTRA Page linked here: https://huggingface.co/transformers/model_doc/electra.html
    labels = torch.LongTensor([sample['labels']]).to(device)
    model_inputs = {k:torch.LongTensor(v).to(device).reshape(1,-1) for k,v in sample.items() if k in model_keys}
    model_output = model(**model_inputs, labels=labels, output_attentions = True)
    sample['loss'] = model_output.loss.item()
    sample['predicted_label'] = model_output.logits[0].argmax().item()
    for i, logit in enumerate(model_output.logits[0]):
        sample[f'logit_{i}'] = logit.item()

    # attentions.append([att.detach() for att in  model_output.attentions])
    
    eval_samples.append(sample)


  0%|          | 0/19666 [00:00<?, ?it/s]

In [ ]:
labels_map = {0: 'entailment', 1: "neutral", 2:"contradiction"}

In [4]:
eval_df = pd.read_parquet(dataset_name + 'eval/eval_df.parquet')
# eval_df = pd.DataFrame(eval_samples)
# eval_df.to_parquet(dataset_name + 'eval/eval_df.parquet')


In [8]:
errors.groupby('label').count()

NameError: name 'errors' is not defined

In [7]:
eval_df.loc[eval_df.premise == 'Two women are embracing while holding to go packages.']

,premise,hypothesis,input_ids,token_type_ids,attention_mask,labels,loss,predicted_label,logit_0,logit_1,logit_2,correct
0,Two women are embracing while holding to go pa...,The sisters are hugging goodbye while holding ...,"[101, 2048, 2308, 2024, 23581, 2096, 3173, 200...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,0.008911,1,-1.701562,3.582637,-1.969213,True
1,Two women are embracing while holding to go pa...,Two woman are holding packages.,"[101, 2048, 2308, 2024, 23581, 2096, 3173, 200...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,0.039708,0,2.316156,-1.212342,-2.179459,True
2,Two women are embracing while holding to go pa...,The men are fighting outside a deli.,"[101, 2048, 2308, 2024, 23581, 2096, 3173, 200...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2,0.001990,2,-3.403698,-2.081107,4.373627,True


In [5]:
eval_df['correct'] = eval_df.labels == eval_df.predicted_label

In [6]:
eval_df.correct.sum() / eval_df.shape[0]

0.8839113190277637

In [35]:
accuracy = (eval_df.labels == eval_df.predicted_label).sum() / eval_df.shape[0]
accuracy

0.8839113190277637

In [37]:
def get_attention(dataset, index, remove_pads=True):
    sample = dataset[index]
    labels = torch.LongTensor([sample['labels']]).to(device)
    model_inputs = {k:torch.LongTensor(v).to(device).reshape(1,-1) for k,v in sample.items() if k in model_keys}
    model_output = model(**model_inputs, labels=labels, output_attentions = True)
    tokens = tokenizer.convert_ids_to_tokens(sample['input_ids'])

    if remove_pads:
        pad_index = next(i for i, v in enumerate(tokens) if v=='[PAD]')
        tokens = tokens[:pad_index]
        attentions = []
        for attention in model_output.attentions:
            attentions.append(attention[:,:,:pad_index,:pad_index])
    hyp_start = next(i for i, v in enumerate(tokens) if v=='[SEP]')
    return tokens, attentions, hyp_start

In [49]:
import plotly.io as pio
pd.options.plotting.backend = "plotly"
pio.templates.default = "plotly_dark"

In [56]:
fig = eval_df.labels.hist(title='Labels distributions')
fig.update_layout(
    # title_text='Sampled Results', # title of plot
    # xaxis_title_text='Value', # xaxis label
    # yaxis_title_text='Count', # yaxis label
    bargap=0.5, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)
fig.show()

In [57]:
errors = eval_df.loc[eval_df.labels != eval_df.predicted_label].sort_values('loss', ascending=False)
fig = errors.labels.hist(title='Labels distributions')
fig.update_layout(
    # title_text='Sampled Results', # title of plot
    # xaxis_title_text='Value', # xaxis label
    # yaxis_title_text='Count', # yaxis label
    bargap=0.5, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)
fig.show()

In [ ]:
errors = eval_df.loc[eval_df.labels != eval_df.predicted_label].sort_values('loss', ascending=False)
fig = errors.labels.hist(title='Labels distributions')
fig.update_layout(
    # title_text='Sampled Results', # title of plot
    # xaxis_title_text='Value', # xaxis label
    # yaxis_title_text='Count', # yaxis label
    bargap=0.5, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)
fig.show()

In [70]:
errors['error_tier'] = 1
errors.loc[errors.loss > errors.loss.quantile(0.666) ,'error_tier'] = 2
errors.loc[errors.loss < errors.loss.quantile(0.33) ,'error_tier'] = 0
errors





,premise,hypothesis,input_ids,token_type_ids,attention_mask,labels,loss,predicted_label,logit_0,logit_1,logit_2,error_tier
12028,"Two skateboarders, one wearing a black t-shirt...",Two snowboarders race.,"[101, 2048, 17260, 6277, 2545, 1010, 2028, 414...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,7.499367,2,-3.512979,-1.581931,3.982009,2
6208,A man wearing chuck taylor shoes and a tan shi...,a man in a straight jacket,"[101, 1037, 2158, 4147, 8057, 4202, 6007, 1998...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,7.182219,2,-3.280823,-1.664646,3.896800,2
17779,An accordion player with a bright colored shir...,A young man is walking.,"[101, 2019, 19060, 2447, 2007, 1037, 4408, 691...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,6.687364,2,-2.864407,-1.903507,3.818442,2
5405,Group of guys sitting in a circle.,There are no females around them.,"[101, 2177, 1997, 4364, 3564, 1999, 1037, 4418...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,6.477018,2,-2.652690,-2.383811,4.090489,2
13857,Two girls kissing a man with a black shirt and...,Two girls kiss.,"[101, 2048, 3057, 7618, 1037, 2158, 2007, 1037...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2,6.406392,0,3.083090,-1.287678,-3.309088,2
...,...,...,...,...,...,...,...,...,...,...,...,...
3747,Several dancers are doing leg lifts on stage.,Performers are doing a show at an event.,"[101, 2195, 10487, 2024, 2725, 4190, 13695, 20...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,0.702682,1,1.292078,1.300625,-3.257061,0
3319,Here are a bunch of people enjoying dinner and...,The group is socializing at the company dinner.,"[101, 2182, 2024, 1037, 9129, 1997, 2111, 9107...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,0.702483,0,1.281285,1.274376,-3.163138,0
730,A little boy bowls in a carpeted hallway.,A boy is rolling a ball in a hotel hallway.,"[101, 1037, 2210, 2879, 15220, 1999, 1037, 101...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,0.699346,2,-3.945910,1.598705,1.607196,0
14742,Five boys in blue shirts drawing at a table.,The boys are working on an art project.,"[101, 2274, 3337, 1999, 2630, 11344, 5059, 201...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,0.699102,0,1.408678,1.404021,-3.518886,0


In [84]:
px.imshow(tiers)

In [82]:
import plotly.express as px
import plotly.graph_objects as go


tiers = errors.reset_index().pivot_table(values='index', index='labels', columns='error_tier', aggfunc='count')
tiers = tiers / tiers.sum(0)

 #aggfunc=lambda x: len(x.unique()))

fig = go.Figure(data=go.Heatmap(
        z=tiers.values,
        x=tiers.index,
        y=tiers.columns,
        colorscale='Viridis'))
fig.show()
# px.imshow(tiers / tiers.sum(0), colorscale='Viridis')

In [91]:
NEGATION_ADVERBS = ["no", "without", "nil","not", "n't", "never", "none", "neith", "nor", "non"]
# NEGATION_VERBS = ["deny", "reject", "refuse", "subside", "retract", "non"]
NEGATIONS = NEGATION_ADVERBS

In [92]:
negs = pd.Series(NEGATIONS)
negs

0         no
1    without
2        nil
3        not
4        n't
5      never
6       none
7      neith
8        nor
9        non
dtype: object

In [134]:
import re

NEG = r"""(?:\b(?:no|not|without|nil|never|none|neith|nor|non)\b)|n't"""

NEG_RE = re.compile(NEG, re.VERBOSE)

def get_count(text):
    count = 0
    for word in text:
        if NEG_RE.search(word):
            count+=1
            continue
        else:
            pass
    return count
# df['text_list'] =  df['text'].apply(lambda x: x.split())
# df['count'] = df['text_list'].apply(lambda x: get_count(x))

In [ ]:
NUMBERS = 



def get_count(text):
    count = 0
    return count

In [165]:
NEG = r"""(?:\b(?:no|not|without|nil|never|none|neith|nor|non)\b)|n't"""

r = re.compile(NEG, flags=re.I | re.X)

In [157]:
pat = re.compile("\bnot\b")

In [181]:
# NEG = r"""(?:\b(?:no|not|without|nil|never|none|neith|nor|non)\b)|n't"""
NEG = r"""(?:\b(?:no|not)\b)|n't"""

r = re.compile(NEG, flags=re.I | re.X)
eval_df['premise_neg_count'] =  eval_df['premise'].apply(lambda x: len(r.findall(x)))
eval_df['hyp_neg_count'] =  eval_df['hypothesis'].apply(lambda x: len(r.findall(x)))
neg_hyp = eval_df.loc[eval_df.hyp_neg_count > 0]

In [185]:
eval_df.loc[eval_df.premise == 'People standing in line of a beer and curry line.']

,premise,hypothesis,input_ids,token_type_ids,attention_mask,labels,loss,predicted_label,logit_0,logit_1,logit_2,correct,premise_neg_count,hyp_neg_count
176,People standing in line of a beer and curry line.,People standing in line,"[101, 2111, 3061, 1999, 2240, 1997, 1037, 5404...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,0.032913,0,2.563461,-0.967239,-2.914764,True,0,0
177,People standing in line of a beer and curry line.,People happily standing in line,"[101, 2111, 3061, 1999, 2240, 1997, 1037, 5404...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,0.013721,1,-1.249188,3.358750,-2.202741,True,0,0
178,People standing in line of a beer and curry line.,People not standing in line,"[101, 2111, 3061, 1999, 2240, 1997, 1037, 5404...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2,0.005238,2,-2.600061,-1.972643,3.704326,True,0,1


In [184]:
neg_hyp

,premise,hypothesis,input_ids,token_type_ids,attention_mask,labels,loss,predicted_label,logit_0,logit_1,logit_2,correct,premise_neg_count,hyp_neg_count
178,People standing in line of a beer and curry line.,People not standing in line,"[101, 2111, 3061, 1999, 2240, 1997, 1037, 5404...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2,0.005238,2,-2.600061,-1.972643,3.704326,True,0,1
280,"Two male police officers on patrol, wearing th...",The officers have shot an unarmed black man an...,"[101, 2048, 3287, 2610, 3738, 2006, 6477, 1010...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2,0.525214,2,-3.139103,1.066642,1.451315,True,0,1
376,A man is sitting in on the side of the street ...,a man does not have brass pots,"[101, 1037, 2158, 2003, 3564, 1999, 2006, 1996...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2,0.014121,2,-3.222080,-1.057600,3.304127,True,0,1
421,A man with a white towel wrapped around the lo...,He doesn't want hair to land on his neck.,"[101, 1037, 2158, 2007, 1037, 2317, 10257, 505...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,0.345027,1,-1.883126,1.339004,0.350566,True,0,1
578,A man is walking away from tents with the word...,The tents do not have anything written on them.,"[101, 1037, 2158, 2003, 3788, 2185, 2013, 1773...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2,0.097265,2,-1.288052,-0.991806,1.845435,True,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19093,A teenage girl in a crowd of people looking up...,no one saw the rainbow in the sky,"[101, 1037, 9454, 2611, 1999, 1037, 4306, 1997...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,4.820199,2,-3.271131,-1.284634,3.526352,False,0,1
19172,A young girl in a pink bikini jumps into the a...,There are no children present.,"[101, 1037, 2402, 2611, 1999, 1037, 5061, 2034...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2,0.003890,2,-3.122002,-1.847906,3.946118,True,0,1
19234,A girl sits at a table in front of a plate of ...,There is no food on the plate.,"[101, 1037, 2611, 7719, 2012, 1037, 2795, 1999...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2,0.003338,2,-3.230085,-1.871974,4.057517,True,0,1
19415,Man balding with a mustache and semi beard sta...,The man has no vocal cords.,"[101, 2158, 13852, 2075, 2007, 1037, 28786, 19...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2,0.828750,2,-0.236021,-0.112475,0.265811,True,0,1


In [182]:
neg_premise = eval_df.loc[eval_df.premise_neg_count > 0]

In [172]:
neg_hyp.correct.sum() / neg_hyp.shape[0]

0.865979381443299

In [176]:
neg_premise.correct.sum() / neg_premise.shape[0]

0.8444444444444444

In [183]:
neg_hyp.correct.sum() / neg_hyp.shape[0], neg_premise.correct.sum() / neg_premise.shape[0]

(0.8571428571428571, 0.8235294117647058)

In [180]:
dd = dataset['validation'].to_pandas()

In [178]:
eval_df.loc[eval_df.labels == 2]

,premise,hypothesis,input_ids,token_type_ids,attention_mask,labels,loss,predicted_label,logit_0,logit_1,logit_2,correct,premise_neg_count,hyp_neg_count
2,Two women are embracing while holding to go pa...,The men are fighting outside a deli.,"[101, 2048, 2308, 2024, 23581, 2096, 3173, 200...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2,0.001990,2,-3.403698,-2.081107,4.373627,True,0,0
5,"Two young children in blue jerseys, one with t...",Two kids in jackets walk to school.,"[101, 2048, 2402, 2336, 1999, 2630, 28772, 101...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2,0.656617,2,-2.897656,1.134209,1.226241,True,0,0
6,A man selling donuts to a customer during a wo...,A woman drinks her coffee in a small cafe.,"[101, 1037, 2158, 4855, 2123, 16446, 2000, 103...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2,0.002018,2,-3.170765,-2.192689,4.331295,True,0,0
10,Two young boys of opposing teams play football...,dog eats out of bowl,"[101, 2048, 2402, 3337, 1997, 10078, 2780, 237...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2,0.002782,2,-2.993377,-2.111204,4.118332,True,0,0
14,A man in a blue shirt standing in front of a g...,A man is wearing a black shirt,"[101, 1037, 2158, 1999, 1037, 2630, 3797, 3061...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2,0.002525,2,-3.310843,-1.994982,4.223023,True,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19652,A boy drags his sled through the snow.,a boy drags his dog,"[101, 1037, 2879, 8011, 2015, 2010, 22889, 209...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2,0.071691,2,-0.989896,-1.505382,2.077693,True,0,0
19655,A man wearing glasses and a ragged costume is ...,Two men are playing frisbee in the park.,"[101, 1037, 2158, 4147, 7877, 1998, 1037, 1420...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2,0.003797,2,-3.818498,-1.512070,4.154389,True,0,0
19658,A female dancer wearing all white dancing whil...,Some players are playing cricket in ground,"[101, 1037, 2931, 8033, 4147, 2035, 2317, 5613...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2,0.001883,2,-3.156940,-2.257585,4.357893,True,0,0
19661,Two women are observing something together.,Two women are standing with their eyes closed.,"[101, 2048, 2308, 2024, 14158, 2242, 2362, 101...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2,0.097676,2,-1.302323,-0.978671,1.842552,True,0,0


In [167]:
r.findall(eval_df.iloc[18573].premise[:-1] + ' not')

["n't", 'not']

In [162]:
k = pat.match(s)
k

In [158]:
s ='i not a'

In [164]:
r.findall(s)

['not']

In [ ]:
re.compile(r'\bAND\b | \bOR\b | \bNOT\b', flags=re.I | re.X)

In [137]:
eval_df.iloc[18573].premise[:-1] + ' not'

"A young Asian child stands at the top of some steps and does n't seem to notice the person taking their picture not"

In [138]:
NEG_RE.search(eval_df.iloc[18573].premise[:-1] + ' not')

In [131]:
for word in eval_df.iloc[18573].premise:
    print(word)

A
 
y
o
u
n
g
 
A
s
i
a
n
 
c
h
i
l
d
 
s
t
a
n
d
s
 
a
t
 
t
h
e
 
t
o
p
 
o
f
 
s
o
m
e
 
s
t
e
p
s
 
a
n
d
 
d
o
e
s
 
n
'
t
 
s
e
e
m
 
t
o
 
n
o
t
i
c
e
 
t
h
e
 
p
e
r
s
o
n
 
t
a
k
i
n
g
 
t
h
e
i
r
 
p
i
c
t
u
r
e
.


In [124]:
tokenizer.convert_tokens_to_ids('no')

2053

In [125]:
o = tokenizer(eval_df.iloc[18573].premise)

In [129]:
tokenizer.convert_ids_to_tokens(o['input_ids'])

['[CLS]',
 'a',
 'young',
 'asian',
 'child',
 'stands',
 'at',
 'the',
 'top',
 'of',
 'some',
 'steps',
 'and',
 'does',
 'n',
 "'",
 't',
 'seem',
 'to',
 'notice',
 'the',
 'person',
 'taking',
 'their',
 'picture',
 '.',
 '[SEP]']

In [122]:
get_count(eval_df.iloc[18573].premise)

0

In [121]:
eval_df.iloc[18573].premise

"A young Asian child stands at the top of some steps and does n't seem to notice the person taking their picture."

In [118]:
eval_df['premise_neg_count'] =  eval_df['premise'].apply(lambda x: get_count(x))
eval_df['hyp_neg_count'] =  eval_df['hypothesis'].apply(lambda x: get_count(x))
eval_df.loc[eval_df.hyp_neg_count > 0]

In [119]:
eval_df.loc[eval_df.hyp_neg_count > 0]

,premise,hypothesis,input_ids,token_type_ids,attention_mask,labels,loss,predicted_label,logit_0,logit_1,logit_2,correct,premise_neg_count,hyp_neg_count


In [116]:
eval_df

,premise,hypothesis,input_ids,token_type_ids,attention_mask,labels,loss,predicted_label,logit_0,logit_1,logit_2,correct,premise_neg_count,hyp_neg_count
0,Two women are embracing while holding to go pa...,The sisters are hugging goodbye while holding ...,"[101, 2048, 2308, 2024, 23581, 2096, 3173, 200...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,0.008911,1,-1.701562,3.582637,-1.969213,True,0,0
1,Two women are embracing while holding to go pa...,Two woman are holding packages.,"[101, 2048, 2308, 2024, 23581, 2096, 3173, 200...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,0.039708,0,2.316156,-1.212342,-2.179459,True,0,0
2,Two women are embracing while holding to go pa...,The men are fighting outside a deli.,"[101, 2048, 2308, 2024, 23581, 2096, 3173, 200...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2,0.001990,2,-3.403698,-2.081107,4.373627,True,0,0
3,"Two young children in blue jerseys, one with t...",Two kids in numbered jerseys wash their hands.,"[101, 2048, 2402, 2336, 1999, 2630, 28772, 101...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,0.028206,0,2.583757,-1.187957,-2.602218,True,0,0
4,"Two young children in blue jerseys, one with t...",Two kids at a ballgame wash their hands.,"[101, 2048, 2402, 2336, 1999, 2630, 28772, 101...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,0.051390,1,-2.916604,2.854484,-0.148932,True,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19661,Two women are observing something together.,Two women are standing with their eyes closed.,"[101, 2048, 2308, 2024, 14158, 2242, 2362, 101...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2,0.097676,2,-1.302323,-0.978671,1.842552,True,0,0
19662,Two women are observing something together.,Two girls are looking at something.,"[101, 2048, 2308, 2024, 14158, 2242, 2362, 101...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,0.092393,0,2.142035,-0.260276,-2.927275,True,0,0
19663,A man in a black leather jacket and a book in ...,A man is flying a kite.,"[101, 1037, 2158, 1999, 1037, 2304, 5898, 6598...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2,0.002443,2,-3.890090,-1.725126,4.397014,True,0,0
19664,A man in a black leather jacket and a book in ...,A man is speaking in a classroom.,"[101, 1037, 2158, 1999, 1037, 2304, 5898, 6598...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,0.018493,0,2.995694,-1.073183,-3.462047,True,0,0


In [111]:
get_count(data.premise)

0

In [106]:
import re
for index, data in eval_df.iterrows():
    count = 0
    for neg in NEGATION_ADVERBS:
        count += len([m.start() for m in re.finditer(neg,data.premise)])
    if count:
        break

In [109]:
data.premise

'A woman is writing something on a post-it note which is hanging on a bulletin board with a lot of other post-it notes.'

In [105]:
index

0

In [95]:
eval_df.premise[0]

'Two women are embracing while holding to go packages.'

In [93]:
negs.isin(eval_df.premise[0])

TypeError: only list-like objects are allowed to be passed to isin(), you passed a [str]

In [89]:
eval_df['p_n_count'] = 

,premise,hypothesis,input_ids,token_type_ids,attention_mask,labels,loss,predicted_label,logit_0,logit_1,logit_2,correct
0,Two women are embracing while holding to go pa...,The sisters are hugging goodbye while holding ...,"[101, 2048, 2308, 2024, 23581, 2096, 3173, 200...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,0.008911,1,-1.701562,3.582637,-1.969213,True
1,Two women are embracing while holding to go pa...,Two woman are holding packages.,"[101, 2048, 2308, 2024, 23581, 2096, 3173, 200...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,0.039708,0,2.316156,-1.212342,-2.179459,True
2,Two women are embracing while holding to go pa...,The men are fighting outside a deli.,"[101, 2048, 2308, 2024, 23581, 2096, 3173, 200...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2,0.001990,2,-3.403698,-2.081107,4.373627,True
3,"Two young children in blue jerseys, one with t...",Two kids in numbered jerseys wash their hands.,"[101, 2048, 2402, 2336, 1999, 2630, 28772, 101...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,0.028206,0,2.583757,-1.187957,-2.602218,True
4,"Two young children in blue jerseys, one with t...",Two kids at a ballgame wash their hands.,"[101, 2048, 2402, 2336, 1999, 2630, 28772, 101...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,0.051390,1,-2.916604,2.854484,-0.148932,True
...,...,...,...,...,...,...,...,...,...,...,...,...
19661,Two women are observing something together.,Two women are standing with their eyes closed.,"[101, 2048, 2308, 2024, 14158, 2242, 2362, 101...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2,0.097676,2,-1.302323,-0.978671,1.842552,True
19662,Two women are observing something together.,Two girls are looking at something.,"[101, 2048, 2308, 2024, 14158, 2242, 2362, 101...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,0.092393,0,2.142035,-0.260276,-2.927275,True
19663,A man in a black leather jacket and a book in ...,A man is flying a kite.,"[101, 1037, 2158, 1999, 1037, 2304, 5898, 6598...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2,0.002443,2,-3.890090,-1.725126,4.397014,True
19664,A man in a black leather jacket and a book in ...,A man is speaking in a classroom.,"[101, 1037, 2158, 1999, 1037, 2304, 5898, 6598...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,0.018493,0,2.995694,-1.073183,-3.462047,True


In [68]:
errors.iloc[2283 - int(2283/3)]

premise            A black and white bird with a yellow beak is f...
hypothesis                       Bird flying outside against the sky
input_ids          [101, 1037, 2304, 1998, 2317, 4743, 2007, 1037...
token_type_ids     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...
attention_mask     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
labels                                                             1
loss                                                        1.232632
predicted_label                                                    0
logit_0                                                     1.543063
logit_1                                                     0.666259
logit_2                                                    -2.944212
Name: 14145, dtype: object

In [ ]:
tokens, attentions, b_start = get_attention(eval_dataset_featurized, 6208)

In [ ]:
model_view(attentions, tokens, b_start)

In [ ]:
head_view(attentions, tokens, b_start)

In [16]:
errors = eval_df.loc[eval_df.labels != eval_df.predicted_label].sort_values('loss', ascending=False)
errors

,premise,hypothesis,input_ids,token_type_ids,attention_mask,labels,loss,predicted_label,logit_0,logit_1,logit_2
1708,,The little girl is planting her first flower.,"[101, 102, 1996, 2210, 2611, 2003, 14685, 2014...","[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...",0,4.945373,1,-2.366696,2.458108,0.338765
6158,,a dog sitting on a bed after ripping up paper.,"[101, 102, 1037, 3899, 3564, 2006, 1037, 2793,...","[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...",0,4.860991,2,-2.651875,0.794354,1.920466
175,,The people are having a food fight.,"[101, 102, 1996, 2111, 2024, 2383, 1037, 2833,...","[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ...",0,4.819573,2,-2.373949,-0.043840,2.350188
4148,,There are only families at the beach today.,"[101, 102, 2045, 2024, 2069, 2945, 2012, 1996,...","[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...",0,4.765400,2,-2.056419,-0.745985,2.668046
5405,,There are no females around them.,"[101, 102, 2045, 2024, 2053, 3801, 2105, 2068,...","[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...",1,4.751141,2,-1.656139,-1.632969,3.100938
...,...,...,...,...,...,...,...,...,...,...,...
1641,,The two people run as quickly as they can for ...,"[101, 102, 1996, 2048, 2111, 2448, 2004, 2855,...","[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2,0.725753,1,-2.539442,1.392833,1.348060
523,,A homeless man has stolen a child's violin in ...,"[101, 102, 1037, 11573, 2158, 2038, 7376, 1037...","[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2,0.723321,1,-2.859130,1.585829,1.538035
5642,,These three cyclists are bitter rivals.,"[101, 102, 2122, 2093, 21912, 2024, 8618, 9169...","[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...",1,0.717352,2,-2.834848,1.521906,1.557448
3322,,"A girl is crying, because she skinned her knee.","[101, 102, 1037, 2611, 2003, 6933, 1010, 2138,...","[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...",1,0.714231,2,-2.365156,1.276089,1.292351


In [19]:
eval_df.loc[eval_df.hypothesis == 'The sisters are hugging goodbye while holding to go packages after just eating lunch.']

,premise,hypothesis,input_ids,token_type_ids,attention_mask,labels,loss,predicted_label,logit_0,logit_1,logit_2
0,,The sisters are hugging goodbye while holding ...,"[101, 102, 1996, 5208, 2024, 17662, 9119, 2096...","[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,0.0865,1,-2.388975,2.648684,0.170106


In [ ]:
errors.iloc[:]

In [ ]:
eval_df.sort_values('loss', ascending=False)

In [ ]:
accuracy = (eval_df.labels == eval_df.predicted_label).sum() / eval_df.shape[0]
accuracy

In [ ]:
type(sample['labels'])

In [ ]:
sample['labels']

In [ ]:
torch.Tensor(sample['labels'])

In [ ]:
# loss = nn.CrossEntropyLoss()
model_keys = ['labels', 'input_ids', 'token_type_ids', 'attention_mask']
# model.to(device)
eval_samples = []
for n, sample in tqdm(enumerate(eval_dataset_featurized), total=len(eval_dataset_featurized)):

    # Change labels depending on whether you're doing NER/Parsing (token classification)
    # or sentiment classification (sequence classification)
    # ELECTRA Page linked here: https://huggingface.co/transformers/model_doc/electra.html
    model_inputs = {k:torch.LongTensor(v).reshape(1,-1) for k,v in sample.items() if k in model_keys}
    model_output = model(**model_inputs)
    sample['loss'] = model_output.loss.item()
    
    eval_samples.append()


In [ ]:
sample

In [ ]:
model_output.loss.item()

In [ ]:
labels

In [ ]:
batch['input_ids']